# Classification d'images avec *tf.keras*

Dans ce projet, on va implémenter des réseaux neuronaux en utilisant les API de Keras.
Nous utiliserons l'implémentation de Keras contenu dans TensorFlow, *tf.keras*.

Lien vers la documentation de [keras.io](https://keras.io/).

Tout le code que nous écrirons dans ce notebook fonctionnera avec la librairie Keras. Nous n'utiliserons pas de lignes spécifiques à TensorFlow.

Pour rappel : la seule différence entre la librairie Keras et le Keras inclus dans TensorFlow est la façon d'importer Keras :

```python
# keras.io code:
from keras.layers import Dense
output_layer = Dense(10)

# corresponding tf.keras code:
from tensorflow.keras.layers import Dense
output_layer = Dense(10)

# or:
from tensorflow import keras
output_layer = keras.layers.Dense(10)
```

#◢  Import & Vérification des versions des librairies

In [ ]:
%matplotlib inline
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
from tensorflow import keras  # tf.keras
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
print("python", sys.version)
for module in tf, keras, pd, np, mpl:
    print(module.__name__, module.__version__)

In [ ]:
assert sys.version_info >= (3, 5) # Python ≥3.5 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

#◢  Chargement du jeu de données MNIST

Keras permet de charger de nombreux datasets avec `keras.datasets`.
Nous utiliserons le jeu de données MNIST *Modifier ou Mixed National Institute of Standards and Technology*, qui est une base de données de chiffres manuscrits.


La base MNIST est devenu un test standard. Elle regroupe 60000 images d'apprentissage et 10000 images de test, issues d'une base de données antérieure, appelée simplement NIST1. 
Ces images sont en noir et blanc, normalisées centrées et de 28 pixels de côté.

Pour l’historique de la création de cette base, voir [Yann LeCun](http://yann.lecun.com/exdb/mnist/).

Séparez le jeu de données d'entrainement en :
- un jeu de validation contenant 5000 images
- un jeu d'entrainement contenant 55000 images

In [ ]:
mnist = keras.datasets.mnist
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

#◢  Exploration des données

Nous avons désormais 3 jeux de données :
- Le jeu de données d'entrainement qui contient 55000 images de tailles 28x28 pixels
- Le jeu de données de validation qui contient 5000 images de tailles 28x28 pixels
- Le jeu de données de test qui contient 10000 images de tailles 28x28 pixels

In [ ]:
#Affichage de la taille du X_train
X_train.shape

In [ ]:
#Affichage de la taille du X_valid
X_valid.shape

In [ ]:
#Affichage de la taille du X_test
X_test.shape

Chaque pixel a une valeur comprise entre 0 et 255

In [ ]:
X_train[0][10:15]

Nous pouvons afficher une image en utilisant la fonction de Matplotlib `imshow()`, avec la carte des couleurs `'binary'` :

In [ ]:
plt.imshow(X_train[1], cmap="binary")
plt.show()

Les labels sont des classes allant de 0 à 9

In [ ]:
y_train[:10]

Regardons quelques images du jeu de données :

In [ ]:
n_rows = 5
n_cols = 10
plt.figure(figsize=(n_cols*2, n_rows*2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title('Chiffre ' + str(y_train[index]))
plt.tight_layout()
plt.show()

#◢  Pré-Processing
Les images étant dans l'échelle [grayscale](https://en.wikipedia.org/wiki/Grayscale), les valeurs varient de 0 à 255.

Afin de normaliser (centrer) les données, nous allons implémnter la fonction Min-Max scaling dans la fonction `normalize_grayscale()`. Après la mise à l'échelle, les valeurs des pixels des données d'entrée iront de 0.1 à 0.9.

Min-Max Scaling:
$
X'=a+{\frac {\left(X-X_{\min }\right)\left(b-a\right)}{X_{\max }-X_{\min }}}
$







In [ ]:
def normalize_grayscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    #Implementation of Min-Max scaling for grayscale image data :

    a = 0.1
    b = 0.9
    image_data_min = image_data.min()
    image_data_max = image_data.max()
    return a + ((image_data - image_data_min)*(b-a))/(image_data_max - image_data_min)

# Test Cases
np.testing.assert_array_almost_equal(
    normalize_grayscale(np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 255])),
    [0.1, 0.103137254902, 0.106274509804, 0.109411764706, 0.112549019608, 0.11568627451, 0.118823529412, 0.121960784314,
     0.125098039216, 0.128235294118, 0.13137254902, 0.9],
    decimal=3)
np.testing.assert_array_almost_equal(
    normalize_grayscale(np.array([0, 1, 10, 20, 30, 40, 233, 244, 254,255])),
    [0.1, 0.103137254902, 0.13137254902, 0.162745098039, 0.194117647059, 0.225490196078, 0.830980392157, 0.865490196078,
     0.896862745098, 0.9])


train_features = normalize_grayscale(X_train)
val_features = normalize_grayscale(X_valid)
test_features = normalize_grayscale(X_test)

print('Tests Passed!')

#◢  Construction d'un modèle 

Construction d'un modèle `Sequential` avec l'API `keras.models.Sequential`, sans aucun argument, et avec 3 couches en utilisant la méthode `add()` :
  * Une couche `Flatten` (`keras.layers.Flatten`) pour convertir chaque image de taille 28x28 image en un simple tableau de 784 pixels. Comme cette couche est la première de votre modèle, vous devez spécifier l'argument `input_shape`.
  * Une couche `Dense` (`keras.layers.Dense`) avec 50 neurones (également appelés units), et la fonction d'activation `"relu"`.
  * Pour finir une couche `Dense` avec 10 neurones (1 par classe), et avec la fonction d'activation `"softmax"` activation pour s'assurer que la somme de toutes les probabilités des classe estimées pour chaque image est égale à 1.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(50, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

Affichons la liste des couches : 

In [ ]:
model.layers

Faisons un `summary` du modèle :

In [ ]:
model.summary()

#◢  Compilation du modèle

Nous utiliserons : 
La fonction de cout `"sparse_categorical_crossentropy"`, l'optimiseur suivant `keras.optimizers.SGD(lr=0.01)`(stochastic gradient descent avec un learning rate à 0.01) et la métrique : `metrics=["accuracy"]`. 

**Note**: Dans la documenation de Keras, vous trouverez d'autres fontions de cout `keras.losses`, d'autres métriques `keras.metrics` et d'autres optimiseurs `keras.optimizer`.

In [1]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.SGD(lr=0.01),
             metrics=["accuracy"])

NameError: ignored

#◢  Entrainement du modèle

Définissons :
- le nombre d'epochs `epochs=5`
- les données de validation `validation_data=(val_features, y_valid)`

**Note**: La méthode `fit()` retourne un objet `History` qui contient les statistiques d'entrainement. N'oubliez pas de récupérer cet objet (`history = model.fit(...)`).

In [ ]:
history = model.fit(train_features, y_train, epochs=5,
                    validation_data=(val_features, y_valid))

Regardons les valeurs de la fonction de cout et de l'accuracy sur les 2 jeux de données (train et validation)

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()

In [ ]:
plot_learning_curves(history)

Relancez `model.fit()` avec 3 epochs, que se passe-t-il ?

In [ ]:
history = model.fit(train_features, y_train, epochs=3,
                    validation_data=(val_features, y_valid))

#◢  Evaluation du modèle


In [ ]:
model.evaluate(test_features, y_test)

#◢  Prédiction


In [ ]:
y_proba = model.predict(test_features)
y_proba[:10].round(3)

A partir des probabilités des différentes valeurs, déduisons-en la valeur prédite (celle qui a le % maximum => utilisez la fonction `argmax`)

In [ ]:
y_pred = y_proba.argmax(axis=1)
y_pred

Utilisons la méthode `predict_classes()` de la classe `model` en passant en paramètre `test_features`.

In [ ]:
y_pred = model.predict_classes(test_features)
y_pred

#◢  Visualisation des prédictions 

Méthodes utilitaires pour afficher une image et un bar chart représentant la probabilité des prédictions pour chaque chiffre de 0 à 9 (les prédictions en bleu sont les prédictions correctes et en rouge celles incorrectes)

In [ ]:
def plot_prediction_image(predictions_array, true_label, img):

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("Chiffre prédit {} {:2.0f}% ({})".format(predicted_label,
                                                        np.max(predictions_array) *100,
                                                        true_label),
                                                        color=color)
    
def plot_prediction_bar_chart(predictions_array, true_label, img):
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    plt.grid(False)
    plt.xticks(range(10))
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

Affichage de l'image à prédire et de la probabilité des prédictions pour chaque chiffre de 0 à 9 pour la première image

In [ ]:
    ### SOLUTION ###
i = 1
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_prediction_image(y_proba[i], y_test[i], X_test[i])
plt.subplot(1,2,2)
plot_prediction_bar_chart(y_proba[i], y_test[i], X_test[i])
plt.show()

## Affichons les 20 premiers chiffres et visualisons les prédictions


In [ ]:
### SOLUTION ###
num_rows = 8
num_cols = 5
plt.figure(figsize=(2*2*num_cols, 2*num_rows))

for row in range(num_rows):
    for col in range(num_cols):
        index = num_cols * row + col
        plt.subplot(num_rows, 2*num_cols, 2*index+1)
        plot_prediction_image(y_proba[index], y_test[index], X_test[index])
        plt.subplot(num_rows, 2*num_cols, 2*index+2)
        plot_prediction_bar_chart(y_proba[index], y_test[index], X_test[index])
plt.tight_layout()
plt.show()



## Focus sur les images mal prédites

In [ ]:
y_proba_false = y_proba[y_pred != y_test]
y_test_false = y_test[y_pred != y_test]
X_test_false = X_test[y_pred != y_test]

In [ ]:
num_rows = 8
num_cols = 5
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for row in range(num_rows):
    for col in range(num_cols):
        index = num_cols * row + col
        plt.subplot(num_rows, 2*num_cols, 2*index+1)
        plot_prediction_image(y_proba_false[index], y_test_false[index], X_test_false[index])
        plt.subplot(num_rows, 2*num_cols, 2*index+2)
        plot_prediction_bar_chart(y_proba_false[index], y_test_false[index], X_test_false[index])
plt.tight_layout()
plt.show()